**The code below to create the 4x4 mapping should create a positive mapping**

In [ ]:
# This code was created by Darshini Rajamani of Purdue University

import random;

def get_random_number():
    return random.randint(-9, 9)

def is_valid(matrix, rows, cols, row, col, num):
    # initial assigment to check validation
    matrix[row][col] = num

    # if row >= 2 and col == cols - 1:  # checking validatation atleast after filling two rows
    #     for i in range(cols):
    #         # check if x + w < 0
    #         if matrix[0][i] is not None and matrix[3][i] is not None:
    #             if matrix[0][i] + matrix[3][i] < 0:
    #                 matrix[row][col] = None  # Undo the initial assignment
    #                 return False

    #         # check if y + z < 0
    #         if matrix[1][i] is not None and matrix[2][i] is not None:
    #             if matrix[1][i] + matrix[2][i] < 0:
    #                 matrix[row][col] = None  # Undo the initial assignment
    #                 return False

    # Check row condition (check in E(2,2))
    if col == 3:
        if sum(matrix[row][:2]) != matrix[row][2] + num:
            return False

    # Check column condition (linearity check)
    if row == 3:
        if matrix[0][col] + matrix[1][col] != num + matrix[2][col]:
            return False

    # Positivity check
    if col == 3:
        matrix[row][col] = num
        if matrix[row][0] + matrix[row][2] < 0:
            matrix[row][col] = None
            return False
        if matrix[row][0] + matrix[row][3] < 0:
            matrix[row][col] = None
            return False
        if matrix[row][1] + matrix[row][2] < 0:
            matrix[row][col] = None
            return False
        if matrix[row][1] + matrix[row][3] < 0:
            matrix[row][col] = None
            return False

    # # Check diagonal condition
    # if row == 3 and col == 3:
    #     if matrix[0][3] + matrix[3][0] != matrix[1][2] + matrix[2][1]:
    #         return False

    # # Check for row uniqueness
    # # Note: this is needed because the rows would repeat without it
    # if col == 3:
    #     matrix[row][col] = num
    #     if tuple(matrix[row]) in set(map(tuple, matrix[:row])):
    #         matrix[row][col] = None
    #         return False
    #     matrix[row][col] = None

    # # Check that only one digit can be 0 in a row (to prevent 0,0,0,0) in a row)
    # if col == 3:
    #   zero_count = sum([1 for x in matrix[row] if x == 0])
    #   if zero_count > 1:
    #     return False
    # if all(x is not None for row in matrix for x in row):
    #     for i in range(cols):
    #         if matrix[1][i] + matrix[2][i] != matrix[0][i] + matrix[3][i]:
    #             return False

    matrix[row][col] = None
    return True

def create_matrix(rows, cols):
    matrix = [[None for _ in range(cols)] for _ in range(rows)]

    backtrackFailedAttempts = 0
    def backtrack(row, col):  # to check for complete and valid solution
        nonlocal backtrackFailedAttempts
        if backtrackFailedAttempts > 1000:
            return False

        if row == rows:
            return True

        nums = [i for i in range(-9, 10)]  # range [-9, 10)
        random.shuffle(nums)

        for num in nums:
            if is_valid(matrix, rows, cols, row, col, num):
                matrix[row][col] = num

                next_row = row
                next_col = col + 1
                if next_col == cols:
                    next_row += 1
                    next_col = 0

                if backtrack(next_row, next_col):
                    return True

        matrix[row][col] = None
        backtrackFailedAttempts = backtrackFailedAttempts + 1
        return False

    backtrack(0, 0)  # recursion

    return matrix


In [14]:
# this code was created by Luke Luschwitz and Karim with very heavy editing by Luke. Abbas and Darshini added some code and influence as well

from scipy.optimize import linprog
import numpy as np

numberOfMappingsToCreate = 12000
#######################
### Create Mappings ###
#######################
listOfMappings = []
for i in range(numberOfMappingsToCreate):
  rows = 4
  cols = 4

  isAValidMatrix = False

  while not isAValidMatrix:
    isAValidMatrix = True
    matrix = create_matrix(rows, cols)
    for row in matrix:
      for element in row:
        if element is None:
          isAValidMatrix = False

  # matrix[1], matrix[3] = matrix[3], matrix[1]

  listOfMappings.append(matrix)


###########################
# Check for Extendability #
###########################
extendableMappings = []
nonExtendableMappings = []

for matrix in listOfMappings:
  mat = [1, 1, 1, 1]
  for_A_ub = [[1,0,0,0],
            [0,1,0,0],
            [0,0,1,0],
            [0,0,0,1],
            [-1,0,0,0],
            [0,-1,0,0],
            [0,0,-1,0],
            [0,0,0,-1],]
  solve_for = [[min(matrix[0][2], matrix[0][3])],
             [min(matrix[1][2], matrix[1][3])],
             [min(matrix[2][2], matrix[2][3])],
             [min(matrix[3][2], matrix[3][3])],
             [-max(-matrix[0][0], -matrix[0][1])],
             [-max(-matrix[1][0], -matrix[1][1])],
             [-max(-matrix[2][0], -matrix[2][1])],
             [-max(-matrix[3][0], -matrix[3][1])]]
  for_A_eq = [[1,1,-1,-1]]
  for_b_eq = [0]

  result = linprog(c=mat, A_ub = for_A_ub, b_ub = solve_for, A_eq = for_A_eq, b_eq = for_b_eq, bounds = None)
  for row in matrix:
    print(row)
  if (result.success):
    extendableMappings.append(matrix)
    print("Extendable")
  else:
    nonExtendableMappings.append(matrix)
    print("Not extendable")
  print()

# Save the sample data to a file
np.save('/content/extendableMappings.npy', extendableMappings)
np.save('/content/nonExtendableMappings.npy', nonExtendableMappings)

Streaming output truncated to the last 5000 lines.
Extendable

[1, 4, 2, 3]
[3, 1, 0, 4]
[4, 4, 2, 6]
[0, 1, 0, 1]
Extendable

[0, 1, 0, 1]
[6, 7, 6, 7]
[4, 2, 3, 3]
[2, 6, 3, 5]
Extendable

[4, 3, 7, 0]
[-3, 9, 3, 3]
[-3, 9, 3, 3]
[4, 3, 7, 0]
Extendable

[4, 3, 5, 2]
[5, 2, 1, 6]
[2, 2, -2, 6]
[7, 3, 8, 2]
Not extendable

[-2, 9, 3, 4]
[8, 3, 2, 9]
[0, 4, 0, 4]
[6, 8, 5, 9]
Extendable

[7, 5, 8, 4]
[-1, 5, 1, 3]
[3, 1, 4, 0]
[3, 9, 5, 7]
Extendable

[3, 8, 7, 4]
[-1, 6, 3, 2]
[-1, 9, 5, 3]
[3, 5, 5, 3]
Extendable

[3, -1, 1, 1]
[3, 0, 2, 1]
[6, -1, 3, 2]
[0, 0, 0, 0]
Extendable

[-1, 6, 2, 3]
[7, 0, 7, 0]
[0, 1, 0, 1]
[6, 5, 9, 2]
Extendable

[9, -2, 4, 3]
[5, 4, 2, 7]
[7, -1, 2, 4]
[7, 3, 4, 6]
Extendable

[0, 3, 0, 3]
[-1, 5, 1, 3]
[0, 5, 0, 5]
[-1, 3, 1, 1]
Extendable

[0, 5, 2, 3]
[8, 9, 8, 9]
[3, 8, 5, 6]
[5, 6, 5, 6]
Extendable

[0, 6, 2, 4]
[2, 4, 2, 4]
[2, 9, 3, 8]
[0, 1, 1, 0]
Extendable

[7, 2, 9, 0]
[2, 9, 4, 7]
[8, 9, 9, 8]
[1, 2, 4, -1]
Not extendable

[-2, 8, 3, 3]
[8, 

In [ ]:
import numpy as np
extendableMappings = np.load('/content/extendableMappings.npy')
nonExtendableMappings = np.load('/content/nonExtendableMappings.npy')

In [15]:
print(f"extendableMappings: {len(extendableMappings)}")
print(f"nonExtendableMappings: {len(nonExtendableMappings)}")

extendableMappings: 9218
nonExtendableMappings: 2782


In [16]:
# All the code below was made by Luke with slight influence from Abbas

import numpy as np

# This function returns the matrix B that is farthest away from the matrices in setA
def farthestBFromA(setB, setA):
  totalDistancesFromA = []
  # iterate through each B matrix
  for B in setB:
    currentBTotalDistance = 0
    # accumulate sum of distances from A to B (distance squared)
    for A in setA:
      currentBTotalDistance = currentBTotalDistance + (np.linalg.norm(np.subtract(B, A)) ** 2)
    totalDistancesFromA.append(currentBTotalDistance)

  orderedBsByDistance = []
  for i in range(len(totalDistancesFromA)):
    index = np.argmax(totalDistancesFromA)
    orderedBsByDistance.append(setB[index])
    totalDistancesFromA[index] = -np.inf
  return orderedBsByDistance



farthestBs = farthestBFromA(nonExtendableMappings, extendableMappings)

for i in range(len(farthestBs[0:3])):
  for row in farthestBs[i]:
      print(row)
  print(f"{i}th farthest from the extendables")
  print()

[5, 3, 9, -1]
[9, 9, 9, 9]
[5, 3, 9, -1]
[9, 9, 9, 9]
0th farthest from the extendables

[2, 6, 9, -1]
[9, 9, 9, 9]
[2, 6, 9, -1]
[9, 9, 9, 9]
1th farthest from the extendables

[9, 0, 9, 0]
[5, 2, 8, -1]
[7, 1, 8, 0]
[7, 1, 9, -1]
2th farthest from the extendables



In [31]:
import numpy as np
# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.metrics import accuracy_score

# Ensure everything is a numpy array
extendableMappings = np.array(extendableMappings)
nonExtendableMappings = np.array(nonExtendableMappings)
farthestBs = np.array(farthestBs)

# Generate input by adding the farthestB matrix to the list of extendableMappings
features = np.concatenate((
    extendableMappings,
    [farthestBs[0]],
    [extendableMappings[0]/1000],
    [nonExtendableMappings[0]/1000]
    ))
print(f"len(features): {len(features)}")

# Flatten the matrices into 1D arrays
features = features.reshape(len(features), -1)

# Generate labels (0 or 1)
labels = [0]*len(extendableMappings) + [1,0,1]
print(f"len(labels): {len(labels)}")
print(f"labels: {labels}")

# Create and train a support vector classifier
model = svm.SVC(kernel='linear', C=1e10, coef0=0.0, tol=1e-5)
model.fit(features, labels)

# Set the bias (intercept) to be zero
model.intercept_ = [0.0]

# Make predictions on the test set
predictions = model.predict(features)

# Evaluate the accuracy of the model
accuracy = accuracy_score(labels, predictions)
print(f"Accuracy: {accuracy}")

# Get the coefficients (weights) of the hyperplane
# coefficients = model.coef_.reshape(4,4)
coefficients = model.coef_

# Intercept of the hyperplane
intercept = model.intercept_

print("Coefficients:\n", coefficients) #the matrix that we get from the ML program
print("Intercept:", intercept)


len(features): 9221
len(labels): 9221
labels: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [35]:
print("coefficients")
print(coefficients.reshape(4,-1))

classifier = np.round(coefficients*10) # this is coefficients, but it is doubled and rounded to integers
print("classifier")
print(classifier.reshape(4,-1))

print()

allMappings = np.concatenate((extendableMappings, nonExtendableMappings), axis=0)
allMappings = allMappings.reshape(len(allMappings), -1)
print()

totalMappingsInClass1 = 0
totalMappingsInClass2 = 0
classifiedMappingResults = []

for i in range(len(allMappings)):
  dotProduct = np.dot(classifier, allMappings[i])
  dotProduct = np.sum(dotProduct)
  extendableOrNot = "E" if i<len(extendableMappings) else "N"
  if dotProduct < 0:
    # print(f"{extendableOrNot} {dotProduct} (class 1)")
    classifiedMappingResults.append((allMappings[i], extendableOrNot, 1))
    totalMappingsInClass1 += 1
  else:
    # print(f"{extendableOrNot} {dotProduct} (class 2)")
    classifiedMappingResults.append((allMappings[i], extendableOrNot, 2))
    totalMappingsInClass2 += 1

for mapping, extendability, _class in classifiedMappingResults:
  if (extendability == "E" and _class == 2):
    print("Bad classifier, extendableMapping in class 2")
    # print(mapping, extendability, _class)
    break
  if totalMappingsInClass1 < 1:
    print("Bad classifier, no mappings in class 1")
    break
  if totalMappingsInClass2 < 1:
    print("Bad classifier, no mappings in class 2")
    break

print(f"totalMappingsInClass1 {totalMappingsInClass1}")
print(f"totalMappingsInClass2 {totalMappingsInClass2}")


coefficients
[[ -66.87792017  -67.2108789    67.04478602 -201.13358509]
 [-100.40051763 -100.06770844   99.89892206 -300.36714813]
 [-233.98971431 -234.32396893  233.82312371 -702.13680694]
 [  66.71127651   67.0453816   -66.87941562  200.63607373]]
classifier
[[ -669.  -672.   670. -2011.]
 [-1004. -1001.   999. -3004.]
 [-2340. -2343.  2338. -7021.]
 [  667.   670.  -669.  2006.]]


totalMappingsInClass1 11551
totalMappingsInClass2 449
